In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import sys

sys.path.insert(0,'/Users/orentapiero/btc_research/')

from btc_functions.glassnode import *
from btc_functions.import_data import get_glassnode_price,get_glassnode_data
from btc_functions.variable_list_urls import *
from btc_functions.utilities import strided_app,strided_app2

plt.rcParams['figure.figsize'] = [15, 10]
sns.set()

In [2]:
all_vars = Urls.keys()

In [3]:
ohlc = get_glassnode_price()
ohlc = ohlc.rename(columns = {'o':'Open','h':'High','l':'Low','c':'Close'})
features = get_glassnode_data(all_vars,Urls)
ohlc['Volume'] = 1e6

  0%|          | 0/173 [00:00<?, ?it/s]

cannot get hash_ribbon.
cannot get difficulty_ribbon.
403 Client Error: Forbidden for url: https://api.glassnode.com/v1/metrics/indicators/nvt_entity_adjusted?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
this asset is forbidden for endpoint

Expecting value: line 1 column 1 (char 0)
cannot get nvt_entity_adjusted.
403 Client Error: Forbidden for url: https://api.glassnode.com/v1/metrics/indicators/spent_output_price_distribution_ath?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
this asset is forbidden for endpoint

Expecting value: line 1 column 1 (char 0)
cannot get spent_output_price_distribution_ath.
403 Client Error: Forbidden for url: https://api.glassnode.com/v1/metrics/indicators/spent_output_price_distribution_percent?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
this asset is forbidden for endpoint

Expecting value: line 1 column 1 (char 0)
cannot get spent_output_price_distribution_percent.
403 Client Error: Forbidden for url: https://api.gla

'DataFrame' object has no attribute 'v'
cannot get futures_open_interest_latest.
'DataFrame' object has no attribute 'v'
cannot get futures_volume_daily_latest.
404 Client Error: Not Found for url: https://api.glassnode.com/v1/metrics/institutions/grayscale_holdings_sum?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
404 page not found

Extra data: line 1 column 5 (char 4)
cannot get grayscale_holdings_sum.
404 Client Error: Not Found for url: https://api.glassnode.com/v1/metrics/institutions/grayscale_flows_sum?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
404 page not found

Extra data: line 1 column 5 (char 4)
cannot get grayscale_flows_sum.
404 Client Error: Not Found for url: https://api.glassnode.com/v1/metrics/institutions/grayscale_premium_percent?a=BTC&i=24h&c=native&api_key=1vUcyF35hTk9awbNGszF0KcLuYH
404 page not found

Extra data: line 1 column 5 (char 4)
cannot get grayscale_premium_percent.
404 Client Error: Not Found for url: https://api.glassnode.com

In [6]:
K = len(features.columns)
L = list(np.arange(K))

from itertools import combinations

Two = [",".join(map(str, comb)) for comb in combinations(L, 2)]
Three = [",".join(map(str, comb)) for comb in combinations(L, 3)]
Four = [",".join(map(str, comb)) for comb in combinations(L, 4)]

print(len(Two)+len(Three)+len(Four))

10676001


In [7]:
import numpy as np

def get_X(data):
    """Return model design matrix X"""
    return data.loc[:,data.columns!='Close'].values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

def feature_selects(List,cols):
    lst = List.split(',')
    lst = [int(l) for l in lst]
    fts = [cols[j] for j in lst]
    return fts

In [56]:
from backtesting import Backtest, Strategy
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

N_TRAIN = 500


class MLTrainOnceStrategy(Strategy):
    price_delta = .99999 # 0.4%

    def init(self):        
        # Init our model, a SVM classifier
        self.clf = SVC()

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        # Skip the training, in-sample data
        if len(self.data) < N_TRAIN:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)
                
        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
#         for trade in self.trades:
#             if current_time - trade.entry_time > pd.Timedelta('2 days'):
#                 if trade.is_long:
#                     trade.sl = max(trade.sl, low)
#                 else:
#                     trade.sl = min(trade.sl, high)

# bt = Backtest(data, 
#               MLTrainOnceStrategy, 
#               commission=.0002, 
#               cash = 10000000,
#               exclusive_orders=True,
#               trade_on_close=True)
# bt.run()
# bt.plot()

In [57]:


def btest(ohlc,features,lst):
    logR = 100*np.log(ohlc['Close']).rename('logR')
    fts = feature_selects(lst,features.columns)
    
    data = pd.concat([ohlc,features[fts]],axis=1)
    data = data.dropna().copy()

    bt = Backtest(data, 
                  MLTrainOnceStrategy, 
                  commission=.0002, 
                  cash = 10000000,
                  exclusive_orders=True,
                  trade_on_close=True)
    bt.run()
    out = dict(metrics = fts, 
               ret = bt._results.loc['Return [%]'],
               sharpe = bt._results.loc['Sharpe Ratio'],
               sortino = bt._results.loc['Sortino Ratio'],
               calmar = bt._results.loc['Calmar Ratio'],
               hit_ratio = bt._results.loc['Win Rate [%]'])
    return out


In [69]:

sam_list = ['puell_multiple', 'sopr_adjusted', 'reserve_risk', 'cdd90_age_adjusted', 
            'average_dormancy', 'liveliness', 'net_unrealized_profit_loss', 'realized_profit_loss_ratio', 
            'realized_profits_to_value_ratio', 'rhodl_ratio', 'cvdd', 'nvts', 
            'marketcap_thermocap_ratio', 'difficulty_latest' ,'mvrv_z_score', 
            'non_zero_count']

L = list(np.arange(len(sam_list)))
ListTwo = [",".join(map(str, comb)) for comb in combinations(L, 3)]

trimmed_features = features[sam_list]
trimmed_features.isna().sum()

puell_multiple                     0
sopr_adjusted                      0
reserve_risk                       0
cdd90_age_adjusted                 0
average_dormancy                   0
liveliness                         0
net_unrealized_profit_loss         0
realized_profit_loss_ratio         0
realized_profits_to_value_ratio    0
rhodl_ratio                        0
cvdd                               0
nvts                               0
marketcap_thermocap_ratio          0
difficulty_latest                  0
mvrv_z_score                       0
non_zero_count                     0
dtype: int64

In [70]:
output = [btest(ohlc,trimmed_features,lst) for lst in tqdm(ListTwo)]
output = pd.DataFrame(output)
output = output.sort_values(by = 'ret',ascending = False)

  0%|          | 0/560 [00:00<?, ?it/s]

In [73]:
output.sort_values(by = 'ret',ascending = False)

,metrics,ret,sharpe,sortino,calmar,hit_ratio
0,"[puell_multiple, sopr_adjusted, reserve_risk]",242.792975,1.017034,1.777024,0.666864,87.5
61,"[puell_multiple, net_unrealized_profit_loss, r...",242.792975,1.017034,1.777024,0.666864,87.5
23,"[puell_multiple, reserve_risk, marketcap_therm...",242.792975,1.017034,1.777024,0.666864,87.5
78,"[puell_multiple, realized_profits_to_value_rat...",242.792975,1.017034,1.777024,0.666864,87.5
79,"[puell_multiple, realized_profits_to_value_rat...",242.792975,1.017034,1.777024,0.666864,87.5
...,...,...,...,...,...,...
59,"[puell_multiple, liveliness, non_zero_count]",-77.912176,0.000000,0.000000,0.000000,12.5
500,"[realized_profit_loss_ratio, marketcap_thermoc...",-77.912176,0.000000,0.000000,0.000000,12.5
501,"[realized_profit_loss_ratio, difficulty_latest...",-77.912176,0.000000,0.000000,0.000000,12.5
502,"[realized_profit_loss_ratio, difficulty_latest...",-77.912176,0.000000,0.000000,0.000000,12.5
